In this notebook, we test out hyperparameter tuning with Optuna. again. with GNBlocks and only 5 edges per node. Also we restrict the range of possible parameter values to more closely match GNBLock_baseline because we know we got not-insane results from there. 

similar to sandbox 8 but with larger dataset and slightly tweaked param ranges.

In [3]:
# ipython extension to autoreload imported modules so that any changes will be up to date before running code in this nb
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from utils.jraph_training import train_and_evaluate_with_data, create_dataset
# from utils.jraph_models import MLPGraphNetwork
from utils.jraph_data import print_graph_fts
import ml_collections
import optuna 
from functools import partial
from datetime import datetime
import os 

/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# set up logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

### set up functions for optuna

In [6]:
CHECKPOINT_PATH = "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/experiments/tuning"

In [7]:
def objective(trial, datasets):
    """ Defines the objective function to be optimized over, aka the validation loss of a model.
    
        Args:
            trial: object which characterizes the current run 
            datasets: dictionary of data. we explicitly pass this in so that we don't have to waste runtime regenerating the same dataset over and over. 
    """
    # create config 
    config = ml_collections.ConfigDict()

    # Optimizer.
    config.optimizer = "adam"
    # config.optimizer = trial.suggest_categorical("optimizer", ["adam", "sgd"])
    config.learning_rate = trial.suggest_float('learning_rate', 1e-4, 5e-2, 
                                               log=True)
    # if config.optimizer == "sgd":
    #     config.momentum = trial.suggest_float('momentum', 0, 0.999) # upper bound is inclusive, and we want to exclude a momentum of 1 because that would yield no decay 

    # Data params that are used in training 
    config.output_steps = 4

    # Training hyperparameters.
    config.batch_size = 1 # variable currently not used
    config.epochs = 10
    config.log_every_epochs = 5
    config.eval_every_epochs = 5
    config.checkpoint_every_epochs = 10

    # GNN hyperparameters.
    config.model = 'MLPBlock'
    config.dropout_rate = trial.suggest_float('dropout_rate', 0, 0.6)
    config.skip_connections = False # This was throwing a broadcast error in add_graphs_tuples_nodes when this was set to True
    config.layer_norm = False # TODO perhaps we want to turn on later

    # choose the hidden layer feature size using powers of 2 
    config.edge_features = (
        2**trial.suggest_int("edge_mlp_1_power", 1, 3), # range 2 - 8; upper bound is inclusive
        2**trial.suggest_int("edge_mlp_2_power", 1, 3), # range 2 - 8
    )
    config.node_features = (
        2**trial.suggest_int("node_mlp_1_power", 1, 8), # range 2 - 512
        2**trial.suggest_int("node_mlp_2_power", 1, 8), # range 2 - 512
        2) 
    # note the last feature size will be the number of features that the graph predicts
    config.global_features = None

    # generate a workdir 
    # TODO: check if we actually care about referencing this in the future or if we can just create a temp dir 
    workdir=os.path.join(CHECKPOINT_PATH, "checkpoints", str(datetime.now()))

    # run training 
    state, train_metrics, eval_metrics_dict = train_and_evaluate_with_data(config=config, workdir=workdir, datasets=datasets)
    
    # retrieve and return val loss (MSE)
    print("eval_metrics_dict['val'].loss", eval_metrics_dict['val'].loss)
    print()
    return eval_metrics_dict['val'].loss.total / eval_metrics_dict['val'].loss.count




In [8]:
def get_data_config():
    config = ml_collections.ConfigDict()

    config.n_samples=10_000
    config.input_steps=1
    config.output_delay=8 # predict 24 hrs into the future 
    config.output_steps=4
    config.timestep_duration=3 # equivalent to 3 hours
    # note a 3 hour timestep resolution would be 5*24/3=40
    # if the time_resolution is 120, then a sampling frequency of 3 would achieve a 3 hour timestep 
    config.sample_buffer = -1 * (config.input_steps + config.output_delay + config.output_steps - 1) # negative buffer so that our sample input are continuous (i.e. the first sample would overlap a bit with consecutive samples) 
        # number of timesteps strictly between the end 
        # of one full sample and the start of the next sample
    config.time_resolution=120 # the number of 
                # raw data points generated per time unit, equivalent to the 
                # number of data points generated per 5 days in the simulation
    config.init_buffer_samples=100
    config.train_pct=0.7
    config.val_pct=0.2
    config.test_pct=0.1
    config.K=36
    config.F=8
    config.c=10
    config.b=10
    config.h=1
    config.seed=42
    config.normalize=True
    config.fully_connected_edges=False

    return config

In [9]:
def prepare_study(study_name):
    # generate dataset 
    dataset_config = get_data_config()
    datasets = create_dataset(dataset_config)
    print_graph_fts(datasets['train']['inputs'][0][0])

    # get the objective function that reuses the pre-generated datasets 
    objective_with_dataset = partial(objective, datasets=datasets)

    # run optimization study
    db_path = os.path.join(CHECKPOINT_PATH, study_name, "optuna_hparam_search.db")
    if not os.path.exists(os.path.join(CHECKPOINT_PATH, study_name)):
        os.makedirs(os.path.join(CHECKPOINT_PATH, study_name))

    study = optuna.create_study(
        study_name=study_name,
        storage=f'sqlite:///{db_path}', # generates a new db if it doesn't exist
        direction='minimize',
        pruner=optuna.pruners.MedianPruner(
            n_startup_trials=5, 
            n_warmup_steps=5
            ), 
        load_if_exists=True, 
    )
    # study.optimize(objective_with_dataset, 
    #                n_trials=25-len(study.trials), 
    #                n_jobs=1)
    
    return study, objective_with_dataset

### try hyperparameter tuning again with fewer params and lower learning rate options

In [10]:
# get study
study9, objective_with_dataset = prepare_study(study_name="hparam_study_9")

Number of nodes: 36
Number of edges: 180
Node features shape: (36, 2)
Edge features shape: (180, 1)
Global features shape: (1, 1)


[I 2023-12-01 00:40:06,037] A new study created in RDB with name: hparam_study_9


In [11]:
study9.optimize(objective_with_dataset, 
                n_trials=5-len(study9.trials), 
                n_jobs=1)

[I 2023-12-01 00:41:50,276] Trial 0 finished with value: 0.9276743531227112 and parameters: {'learning_rate': 0.0028473673130245706, 'dropout_rate': 0.5273369390473157, 'edge_mlp_1_power': 3, 'edge_mlp_2_power': 1, 'node_mlp_1_power': 8, 'node_mlp_2_power': 4}. Best is trial 0 with value: 0.9276743531227112.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1855.3488, dtype=float32), count=Array(2000., dtype=float32))



[I 2023-12-01 00:42:55,181] Trial 1 finished with value: 0.9276743531227112 and parameters: {'learning_rate': 0.028485552089892174, 'dropout_rate': 0.07430617255491741, 'edge_mlp_1_power': 1, 'edge_mlp_2_power': 1, 'node_mlp_1_power': 5, 'node_mlp_2_power': 1}. Best is trial 0 with value: 0.9276743531227112.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1855.3488, dtype=float32), count=Array(2000., dtype=float32))



[I 2023-12-01 00:45:11,189] Trial 2 finished with value: 0.8017078638076782 and parameters: {'learning_rate': 0.0009191923909642663, 'dropout_rate': 0.11729063791922098, 'edge_mlp_1_power': 2, 'edge_mlp_2_power': 1, 'node_mlp_1_power': 3, 'node_mlp_2_power': 5}. Best is trial 2 with value: 0.8017078638076782.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1603.4158, dtype=float32), count=Array(2000., dtype=float32))



[I 2023-12-01 00:51:13,006] Trial 3 finished with value: 0.9276743531227112 and parameters: {'learning_rate': 0.00022048350903653192, 'dropout_rate': 0.33185884038643293, 'edge_mlp_1_power': 1, 'edge_mlp_2_power': 2, 'node_mlp_1_power': 8, 'node_mlp_2_power': 8}. Best is trial 2 with value: 0.8017078638076782.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1855.3488, dtype=float32), count=Array(2000., dtype=float32))



[I 2023-12-01 00:52:24,166] Trial 4 finished with value: 0.9276743531227112 and parameters: {'learning_rate': 0.04067520627088882, 'dropout_rate': 0.15582946788750354, 'edge_mlp_1_power': 2, 'edge_mlp_2_power': 3, 'node_mlp_1_power': 2, 'node_mlp_2_power': 6}. Best is trial 2 with value: 0.8017078638076782.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1855.3488, dtype=float32), count=Array(2000., dtype=float32))



still i dont understand how some trials can end up with literally the exact same validation error to like 10 decimal places. 

In [12]:
study9.trials

[FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.9276743531227112], datetime_start=datetime.datetime(2023, 12, 1, 0, 40, 6, 147970), datetime_complete=datetime.datetime(2023, 12, 1, 0, 41, 50, 257574), params={'learning_rate': 0.0028473673130245706, 'dropout_rate': 0.5273369390473157, 'edge_mlp_1_power': 3, 'edge_mlp_2_power': 1, 'node_mlp_1_power': 8, 'node_mlp_2_power': 4}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.05, log=True, low=0.0001, step=None), 'dropout_rate': FloatDistribution(high=0.6, log=False, low=0.0, step=None), 'edge_mlp_1_power': IntDistribution(high=3, log=False, low=1, step=1), 'edge_mlp_2_power': IntDistribution(high=3, log=False, low=1, step=1), 'node_mlp_1_power': IntDistribution(high=8, log=False, low=1, step=1), 'node_mlp_2_power': IntDistribution(high=8, log=False, low=1, step=1)}, trial_id=1, value=None),
 FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.9276743531227

ok let's try visualizing this to see what in the hell is going wrong

In [13]:
fig = optuna.visualization.plot_intermediate_values(study9)
fig.show()

[W 2023-12-01 00:52:24,604] You need to set up the pruning feature to utilize `plot_intermediate_values()`


In [14]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study9, params=['learning_rate', 'dropout_rate'])
fig.show()

In [15]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study9, params=['learning_rate', 'node_mlp_1_power'])
fig.show()

ok literally why is this obsessed with the number 22.

In [16]:
study9.optimize(objective_with_dataset, 
                n_trials=50-len(study9.trials), 
                n_jobs=1)

[I 2023-12-01 00:53:22,903] Trial 5 finished with value: 0.8474153280258179 and parameters: {'learning_rate': 0.0006210059574715593, 'dropout_rate': 0.3696856038938989, 'edge_mlp_1_power': 3, 'edge_mlp_2_power': 2, 'node_mlp_1_power': 3, 'node_mlp_2_power': 6}. Best is trial 2 with value: 0.8017078638076782.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1694.8307, dtype=float32), count=Array(2000., dtype=float32))



[I 2023-12-01 00:54:44,753] Trial 6 finished with value: 0.8673476576805115 and parameters: {'learning_rate': 0.000377598608437538, 'dropout_rate': 0.4500927410381871, 'edge_mlp_1_power': 2, 'edge_mlp_2_power': 3, 'node_mlp_1_power': 7, 'node_mlp_2_power': 5}. Best is trial 2 with value: 0.8017078638076782.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1734.6953, dtype=float32), count=Array(2000., dtype=float32))



[I 2023-12-01 00:56:03,275] Trial 7 finished with value: 0.7926386594772339 and parameters: {'learning_rate': 0.00021969918188829144, 'dropout_rate': 0.14863811410941025, 'edge_mlp_1_power': 1, 'edge_mlp_2_power': 3, 'node_mlp_1_power': 7, 'node_mlp_2_power': 3}. Best is trial 7 with value: 0.7926386594772339.


...

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1551.4808, dtype=float32), count=Array(2000., dtype=float32))



[I 2023-12-01 01:46:28,279] Trial 48 finished with value: 0.6886663436889648 and parameters: {'learning_rate': 0.0008264309017836939, 'dropout_rate': 0.02797797230322352, 'edge_mlp_1_power': 3, 'edge_mlp_2_power': 3, 'node_mlp_1_power': 3, 'node_mlp_2_power': 8}. Best is trial 34 with value: 0.6736546754837036.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1377.3326, dtype=float32), count=Array(2000., dtype=float32))



[I 2023-12-01 01:47:27,148] Trial 49 finished with value: 0.7413574457168579 and parameters: {'learning_rate': 0.0008148921757940187, 'dropout_rate': 0.09909936971259536, 'edge_mlp_1_power': 3, 'edge_mlp_2_power': 3, 'node_mlp_1_power': 3, 'node_mlp_2_power': 4}. Best is trial 34 with value: 0.6736546754837036.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1482.7148, dtype=float32), count=Array(2000., dtype=float32))



In [17]:
fig = optuna.visualization.plot_intermediate_values(study9)
fig.show()

[W 2023-12-01 01:47:27,260] You need to set up the pruning feature to utilize `plot_intermediate_values()`


In [18]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study9, params=['learning_rate', 'dropout_rate'])
fig.show()

In [19]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study9, params=['learning_rate', 'edge_mlp_1_power'])
fig.show()

In [20]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study9, params=['learning_rate', 'node_mlp_1_power'])
fig.show()

In [21]:
print(study9.direction)

StudyDirection.MINIMIZE


In [22]:
study9.trials

[FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.9276743531227112], datetime_start=datetime.datetime(2023, 12, 1, 0, 40, 6, 147970), datetime_complete=datetime.datetime(2023, 12, 1, 0, 41, 50, 257574), params={'learning_rate': 0.0028473673130245706, 'dropout_rate': 0.5273369390473157, 'edge_mlp_1_power': 3, 'edge_mlp_2_power': 1, 'node_mlp_1_power': 8, 'node_mlp_2_power': 4}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.05, log=True, low=0.0001, step=None), 'dropout_rate': FloatDistribution(high=0.6, log=False, low=0.0, step=None), 'edge_mlp_1_power': IntDistribution(high=3, log=False, low=1, step=1), 'edge_mlp_2_power': IntDistribution(high=3, log=False, low=1, step=1), 'node_mlp_1_power': IntDistribution(high=8, log=False, low=1, step=1), 'node_mlp_2_power': IntDistribution(high=8, log=False, low=1, step=1)}, trial_id=1, value=None),
 FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.9276743531227

In [23]:
study9.optimize(objective_with_dataset, 
                n_trials=200-len(study9.trials), 
                n_jobs=1)

[I 2023-12-01 01:48:27,737] Trial 50 finished with value: 0.9276743531227112 and parameters: {'learning_rate': 0.00029235578141400685, 'dropout_rate': 0.021329120538731573, 'edge_mlp_1_power': 3, 'edge_mlp_2_power': 3, 'node_mlp_1_power': 3, 'node_mlp_2_power': 5}. Best is trial 34 with value: 0.6736546754837036.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1855.3488, dtype=float32), count=Array(2000., dtype=float32))



[I 2023-12-01 01:49:54,668] Trial 51 finished with value: 0.7484281659126282 and parameters: {'learning_rate': 0.0005980161100761534, 'dropout_rate': 0.06053792482117635, 'edge_mlp_1_power': 3, 'edge_mlp_2_power': 3, 'node_mlp_1_power': 4, 'node_mlp_2_power': 8}. Best is trial 34 with value: 0.6736546754837036.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1496.8563, dtype=float32), count=Array(2000., dtype=float32))



[I 2023-12-01 01:51:16,601] Trial 52 finished with value: 0.6965382099151611 and parameters: {'learning_rate': 0.0012092622387119899, 'dropout_rate': 0.02831642781445371, 'edge_mlp_1_power': 3, 'edge_mlp_2_power': 1, 'node_mlp_1_power': 3, 'node_mlp_2_power': 8}. Best is trial 34 with value: 0.6736546754837036.


...

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1365.3551, dtype=float32), count=Array(2000., dtype=float32))



[I 2023-12-01 04:41:32,734] Trial 198 finished with value: 0.6931079626083374 and parameters: {'learning_rate': 0.0005171550144509658, 'dropout_rate': 0.031225638520034855, 'edge_mlp_1_power': 2, 'edge_mlp_2_power': 2, 'node_mlp_1_power': 6, 'node_mlp_2_power': 5}. Best is trial 84 with value: 0.6667227149009705.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1386.216, dtype=float32), count=Array(2000., dtype=float32))



[I 2023-12-01 04:42:44,686] Trial 199 finished with value: 0.6751635074615479 and parameters: {'learning_rate': 0.0008277710764148207, 'dropout_rate': 0.001805602796220595, 'edge_mlp_1_power': 2, 'edge_mlp_2_power': 2, 'node_mlp_1_power': 6, 'node_mlp_2_power': 5}. Best is trial 84 with value: 0.6667227149009705.


eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(1350.327, dtype=float32), count=Array(2000., dtype=float32))



In [24]:
study9.trials

[FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.9276743531227112], datetime_start=datetime.datetime(2023, 12, 1, 0, 40, 6, 147970), datetime_complete=datetime.datetime(2023, 12, 1, 0, 41, 50, 257574), params={'learning_rate': 0.0028473673130245706, 'dropout_rate': 0.5273369390473157, 'edge_mlp_1_power': 3, 'edge_mlp_2_power': 1, 'node_mlp_1_power': 8, 'node_mlp_2_power': 4}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.05, log=True, low=0.0001, step=None), 'dropout_rate': FloatDistribution(high=0.6, log=False, low=0.0, step=None), 'edge_mlp_1_power': IntDistribution(high=3, log=False, low=1, step=1), 'edge_mlp_2_power': IntDistribution(high=3, log=False, low=1, step=1), 'node_mlp_1_power': IntDistribution(high=8, log=False, low=1, step=1), 'node_mlp_2_power': IntDistribution(high=8, log=False, low=1, step=1)}, trial_id=1, value=None),
 FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.9276743531227

In [25]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study9, params=['learning_rate', 'dropout_rate'])
fig.show()

In [26]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study9, params=['learning_rate', 'edge_mlp_1_power'])
fig.show()

In [27]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study9, params=['learning_rate', 'node_mlp_1_power'])
fig.show()